# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
g_key = os.environ.get("GKEY")

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/WeatherPy_CSV.csv')
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Derzhavīnsk,51.10,66.32,39.79,69,0,10.56,KZ,1602994267
1,1,Cap Malheureux,-19.98,57.61,78.01,63,24,8.99,MU,1602994267
2,2,Finspång,58.71,15.77,39.20,74,51,10.29,SE,1602994267
3,3,Prachuap Khiri Khan,12.22,99.75,77.00,88,75,5.82,TH,1602993983
4,4,Andenes,69.31,16.12,35.60,93,75,20.80,NO,1602994268


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coor = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df['Humidity'].astype(int)

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [5]:

fig = gmaps.figure(layout=figure_layout)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coor, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)


# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 80 degrees but higher than 70
* Wind speed less than 10 mph.
* Zero cloudiness.

In [6]:
cities_df = weather_df.copy()
cities_df = cities_df[(cities_df['Max Temp']>=70) & (cities_df['Max Temp']<=80) 
                      & (cities_df['Wind Speed']<=10) &(cities_df['Cloudiness']==0)]

cities_df=cities_df.reset_index(drop=True)
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,104,Taoudenni,22.68,-3.98,79.02,22,0,6.31,ML,1602994290
1,110,Binga,-17.62,27.34,71.40,40,0,0.60,ZW,1602994291
2,123,Rancho Palos Verdes,33.74,-118.39,73.00,64,0,5.32,US,1602994267
3,142,Ubari,26.59,12.78,71.83,22,0,2.77,LY,1602994297
4,217,Muscat,23.61,58.59,73.40,56,0,3.36,OM,1602994298


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# copy a data frame name hotel_df
hotel_df = cities_df.copy()

In [8]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#checker variable
i = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][i])+' , '+str(hotel_df['Lng'][i])
    i+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)
hotels

[nan,
 'Lakeview Inn Binga',
 'Terranea Resort',
 'فندق أوباري',
 'Mutrah Hotel',
 nan,
 'Nuba Nile',
 'Holiday Inn Express & Suites Elk Grove Central - Hwy 99',
 nan,
 'Conquistador Hotel',
 'Observatory Guest House']

In [9]:
# adding hotel name colum to hotel_df
hotel_df['Hotel Name'] = hotels

In [10]:
# drop cities without hotel within 5000 m
hotel_df_clean = hotel_df.dropna()

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

heat_layer = gmaps.heatmap_layer(coor, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…